In [57]:
import torch
torch.cuda.is_available()

True

In [58]:
# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    device = torch.device("cuda")
else:
    print("CUDA is not available. Using CPU.")
    device = torch.device("cpu")

CUDA is available. Using GPU.


In [59]:
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

In [60]:
file_path = 'D:\MachineLearning\MachineLearningHw\Hw6\dataset\Housing-pre.csv'
housing = pd.read_csv(file_path)
housing.head

<bound method NDFrame.head of        price  area  bedrooms  bathrooms  stories  mainroad  guestroom  \
0    3290000  3792         4          1        2         1          0   
1    3290000  4040         2          1        1         1          0   
2    3290000  2145         3          1        2         1          0   
3    3290000  5880         3          1        1         1          0   
4    3255000  4500         2          1        1         0          0   
..       ...   ...       ...        ...      ...       ...        ...   
106  1820000  3000         2          1        1         1          0   
107  1767150  2400         3          1        1         0          0   
108  1750000  3620         2          1        1         1          0   
109  1750000  2910         3          1        1         0          0   
110  1750000  3850         3          1        2         1          0   

     basement  hotwaterheating  airconditioning  parking  prefarea  \
0           0          

In [61]:
df_train, df_test = train_test_split(housing, train_size = 0.8, test_size = 0.2, random_state = 30)

In [62]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# Apply scaler() to all the columns except the 'yes-no' and 'dummy' variables
num_vars = ['price','area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking','prefarea']

df_train[num_vars] = scaler.fit_transform(df_train[num_vars])
df_test[num_vars] = scaler.transform(df_test[num_vars])
df_train.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
88,0.409091,0.112245,0.50,0.0,0.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
98,0.250000,0.183673,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
78,0.500000,0.183673,0.25,0.0,0.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
70,0.586364,0.208844,0.25,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
103,0.090909,0.006803,0.50,0.0,0.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
# Separate features and labels
y_train= df_train[num_vars].values[:, 0]  # get input values from first column -- y is a list here
x_train = df_train[num_vars].values[:, 1:13]  # get output values from second column -- x is the list here
x_test = df_test[num_vars].values[:, 1:]  
y_test = df_test[num_vars].values[:, 0]  
m = len(y_train)  # Number of training examples
n = len(x_train)  # Number of training examples

In [64]:
print(x_train.shape)  

t_c_train = torch.tensor(x_train, dtype=torch.float32)  
t_u_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
t_c_val = torch.tensor(x_test, dtype=torch.float32)  
t_u_val = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

(88, 11)


In [65]:
import torch.nn as nn

In [66]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_c_train, t_c_val, t_u_train, t_u_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_c_train) 
        loss_train = loss_fn(t_p_train, t_u_train)  

        t_p_val = model(t_c_val)  
        loss_val = loss_fn(t_p_val, t_u_val)  

        optimizer.zero_grad()
        loss_train.backward()  
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f}, Validation loss {loss_val.item():.4f}")


In [67]:
seq_model = nn.Sequential(
    nn.Linear(11, 32),
    nn.Tanh(),
    nn.Linear(32, 1))
optimizer = optim.SGD(seq_model.parameters(), lr=1e-3)
training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),  
    t_c_train = t_c_train,   
    t_c_val = t_c_val,       
    t_u_train = t_u_train,  
    t_u_val = t_u_val       
)



Epoch 1, Training loss 0.6135, Validation loss 0.5854
Epoch 1000, Training loss 0.0779, Validation loss 0.0684
Epoch 2000, Training loss 0.0756, Validation loss 0.0672
Epoch 3000, Training loss 0.0739, Validation loss 0.0664
Epoch 4000, Training loss 0.0727, Validation loss 0.0658
Epoch 5000, Training loss 0.0717, Validation loss 0.0655


In [68]:
#Problem 1 b)

In [69]:
seq_model = nn.Sequential(
    nn.Linear(11, 32),
    nn.Tanh(),
    nn.Linear(32, 64),
    nn.Tanh(),
    nn.Linear(64, 16),
    nn.Tanh(),
    nn.Linear(16,1))

optimizer = optim.SGD(seq_model.parameters(), lr=1e-3)
training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),  
    t_c_train = t_c_train,   
    t_c_val = t_c_val,       
    t_u_train = t_u_train,  
    t_u_val = t_u_val       
)


Epoch 1, Training loss 0.2611, Validation loss 0.2322
Epoch 1000, Training loss 0.0746, Validation loss 0.0587
Epoch 2000, Training loss 0.0741, Validation loss 0.0588
Epoch 3000, Training loss 0.0737, Validation loss 0.0587
Epoch 4000, Training loss 0.0734, Validation loss 0.0586
Epoch 5000, Training loss 0.0731, Validation loss 0.0585


In [83]:
#Problem 2 a)
from torchvision import datasets, transforms
transform = transforms.Compose([
    transforms.ToTensor(), 
])
print('test')

test


In [75]:


data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True, transform=transform) 
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True, transform=transform) 

Files already downloaded and verified
Files already downloaded and verified


In [76]:
class FCNN(nn.Module):
    def __init__(self):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(3*32*32, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = x.view(-1, 3*32*32)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [77]:
# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    device = torch.device("cuda")
else:
    print("CUDA is not available. Using CPU.")
    device = torch.device("cpu")

CUDA is available. Using GPU.


In [78]:
model = FCNN().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

In [79]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

In [80]:
def train_model(n_epochs, train_loader, valid_loader, model, optimizer, loss_fn, device):
    start = time.time()
    for epoch in range(n_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.view(imgs.shape[0], -1).to(device)  # Flatten and send to device
            labels = labels.to(device)

            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_loss = running_loss / len(train_loader)
        print(f"Epoch: {epoch}, Training Loss: {avg_loss}")

      
        model.eval()  
        total = 0
        correct = 0
        with torch.no_grad(): 
            for imgs, labels in valid_loader:
                imgs = imgs.view(imgs.shape[0], -1).to(device)
                labels = labels.to(device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f'Validation Accuracy after epoch {epoch}: {accuracy}%')

    stop = time.time()
    duration = stop - start
    print(f'Total Training Time: {duration} seconds')

In [82]:
train_model(100, train_loader, valid_loader, model, optimizer, loss_fn, device)

Epoch: 0, Training Loss: 1.5200012658563111
Validation Accuracy after epoch 0: 44.4%
Epoch: 1, Training Loss: 1.4942895587143081
Validation Accuracy after epoch 1: 47.88%
Epoch: 2, Training Loss: 1.4640917290202187
Validation Accuracy after epoch 2: 46.69%
Epoch: 3, Training Loss: 1.4373683340256782
Validation Accuracy after epoch 3: 45.85%
Epoch: 4, Training Loss: 1.4217397686465623
Validation Accuracy after epoch 4: 47.0%
Epoch: 5, Training Loss: 1.3970257896748954
Validation Accuracy after epoch 5: 46.39%
Epoch: 6, Training Loss: 1.3807187626124038
Validation Accuracy after epoch 6: 48.53%
Epoch: 7, Training Loss: 1.3670200605679046
Validation Accuracy after epoch 7: 47.87%
Epoch: 8, Training Loss: 1.3430477565206835
Validation Accuracy after epoch 8: 47.61%
Epoch: 9, Training Loss: 1.3314245820350354
Validation Accuracy after epoch 9: 48.14%
Epoch: 10, Training Loss: 1.3101317935130175
Validation Accuracy after epoch 10: 49.69%
Epoch: 11, Training Loss: 1.305632758323494
Validation

In [84]:
#Problem 2b)
class FCNN(nn.Module):
    def __init__(self):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(3*32*32, 1024)  # First hidden layer
        self.fc2 = nn.Linear(1024, 512)      # Second hidden layer
        self.fc3 = nn.Linear(512, 256)       # Third hidden layer
        self.fc4 = nn.Linear(256, 10)        # Output layer

    def forward(self, x):
        x = x.view(-1, 3*32*32)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [85]:
model = FCNN().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

In [86]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

In [87]:
def train_model(n_epochs, train_loader, valid_loader, model, optimizer, loss_fn, device):
    start = time.time()
    for epoch in range(n_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.view(imgs.shape[0], -1).to(device)  # Flatten and send to device
            labels = labels.to(device)

            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_loss = running_loss / len(train_loader)
        print(f"Epoch: {epoch}, Training Loss: {avg_loss}")

      
        model.eval()  
        total = 0
        correct = 0
        with torch.no_grad(): 
            for imgs, labels in valid_loader:
                imgs = imgs.view(imgs.shape[0], -1).to(device)
                labels = labels.to(device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f'Validation Accuracy after epoch {epoch}: {accuracy}%')

    stop = time.time()
    duration = stop - start
    print(f'Total Training Time: {duration} seconds')

In [88]:
train_model(300, train_loader, valid_loader, model, optimizer, loss_fn, device)

Epoch: 0, Training Loss: 1.9241075677335109
Validation Accuracy after epoch 0: 36.37%
Epoch: 1, Training Loss: 1.6968997401349686
Validation Accuracy after epoch 1: 41.11%
Epoch: 2, Training Loss: 1.6030331502485153
Validation Accuracy after epoch 2: 43.05%
Epoch: 3, Training Loss: 1.531828205146448
Validation Accuracy after epoch 3: 46.7%
Epoch: 4, Training Loss: 1.4774206751752692
Validation Accuracy after epoch 4: 45.79%
Epoch: 5, Training Loss: 1.4359060082289263
Validation Accuracy after epoch 5: 47.57%
Epoch: 6, Training Loss: 1.3917195341166329
Validation Accuracy after epoch 6: 49.97%
Epoch: 7, Training Loss: 1.3470011172087297
Validation Accuracy after epoch 7: 50.14%
Epoch: 8, Training Loss: 1.3236469288006463
Validation Accuracy after epoch 8: 49.76%
Epoch: 9, Training Loss: 1.2851266159730799
Validation Accuracy after epoch 9: 48.46%
Epoch: 10, Training Loss: 1.2513645731122292
Validation Accuracy after epoch 10: 50.31%
Epoch: 11, Training Loss: 1.2273028717778833
Validatio